# GEOG5990M Portfolio Assignment 2

Student ID number: 201682556


## Submission
In my final project I will be assessing whether the number of burglaries within key city’s is correlated to the distance of the property to what is considered a primary road, for example a motorway junction. The idea being investigated is that burglars are more likely to target properties and areas that have excellent highway connections to a primary road allowing them to both get away quickly as well as disguise themselves amongst a large amount of traffic making it less likely the police will be able to identify their vehicle in a sufficient time. The output areas I have chosen for this investigation are prominent cities in the North of England; Leeds, Bradford, Sheffield, and Hull.



In [ ]:
#This file was made in google colab

#first use pip for contextily and geoplot packages
!pip install contextily
!pip install geoplot

#Import the required packages
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

import pyproj
import contextily as ctx
import seaborn as sns
#create points from excel
import fiona
#from collections import OrderedDict

import geoplot as gplt
import geoplot.crs as gcrs

Preprocessing:

The Census data was taken from a bulk download of 2021 from the NOMIS webpage (Nomis, 2021). The chosen category’s which would have the most influence was TS006: Population density, TS017: Household size and TS050 number of bedrooms. As each category is its own excel sheet, I elected to merge all of them together before preprocessing using python. I also formatted the household size into a simple two columns; properties with up to three people and properties more than 3 people, reducing from the original eight columns.

The Crime data was downloaded using a custom data download widget, specifying February 2023 – February 2024, this was the latest data available and I wanted to analyse the data over a one-year period (data.police, no date). This download was separated by month so I combined all twelve sheets into one spreadsheet. Originally, I was going to convert the excel file into a geo-frame within python, however I was not able to read the geo-frame in further command lines so I opted to add the excel file to QGIS and create the geometry, saving as a shapefile. Since I was creating a new shapefile, I also converted the results into British National Grid (BNG) from latitude longitude.

I downloaded the road network data from Digimaps using the BNG tile function, SE, SK, and TA (digimap, 2023). As each of these contained 2 separate shapefiles for network and nodes, I merged all the Road_Link shapefiles together however this was a large file at 500MB so I decided to condense this further before processing in python by selecting all roads that intersect inside or out, the target cities. In addition to the road data, I also created an ad-hoc standalone shapefile containing the starting points of all the roads where primary equals true i.e. Motorways, major trunk roads or arterial roads.

Output areas (polygons) were downloaded from Geoportal (Geoportal Statistics, 2023), I went with the Dec 2021 download to match as close as possible the recorded census data as the LSOA codes generally has slight variation between years. I filtered this output using the website interactive mapping tool and drew a box around my area of interest. This allowed me to download a much smaller area than the original entire country’s worth of areas, I also manually edited this in QGIS to further remove extraneous polygons that were not in the study areas making the large file size as small as possible.


In [ ]:
# Data-preprocessing

#Import shapefiles

## the dbf/shp may take several minutes to upload if using google colab

#Excel
census = pd.read_excel("census_merged.xlsx")

#crime Points
crimes_merged = gpd.read_file("crimes_merged.shp")

#road Points
road_start_points = gpd.read_file("primary_roads_start_points.shp")

#Lines
roads = gpd.read_file("major_roads.shp")
#Polygons
LSOA = gpd.read_file("Output_Areas_Dec_2021_v2.shp")

# Old excel file for crimes
#crimes = pd.read_excel("crimes_merged.xlsx")

In [ ]:
LSOA.explore()

In [ ]:
LSOA.dtypes

In [ ]:
roads.dtypes

In [ ]:
roads.head()

I clean the Output areas to only include the chosen city’s using a string contains function to find key words and text as the output areas download does not have a standalone column with city names by default. Using the explore function you can visualise the area of my investigation clearly, this file with be used as an important template for other datasets later. Checking the coordinate systems are all the same.


In [ ]:
#filter LSOA to target areas within the City Limits
#Selecting rows IF contains city

#Only returns a maximum of two cities to display
#lsoa_cleaned = LSOA.loc[LSOA["lsoa11nm"].str.contains("Leeds | Bradford | Hull | Sheffield")]

# Use long code instead of concatenating into one bracket to fix above issue.
lsoa_cleaned = LSOA.loc[LSOA["LSOA21NM"].str.contains("Leeds") | (LSOA["LSOA21NM"].str.contains("Bradford") | (LSOA["LSOA21NM"].str.contains("Hull") | (LSOA["LSOA21NM"].str.contains("Sheffield") )))]

lsoa_cleaned.explore(width="80%", height="60%")

Clip all roads by poylgons then filter to make primary

In [ ]:
#make sure CRS are the same
print('lsoa_cleaned.crs:', lsoa_cleaned.crs)
print('roads.crs:', roads.crs)
print('crimes.crs:', roads.crs)

In [ ]:
#coord systems coversion (no longer needed)
#lsoa_cleaned = lsoa_cleaned.to_crs(epsg=27700) #BNG

#make sure dataframes CRS are the same
#print('lsoa_cleaned.crs:', lsoa_cleaned.crs)
#print('roads.crs:', roads.crs)

I create a dissolved polygon of the city’s and then clipped the road links shapefile using this polygon to only use the relevant roads located in and adjoining the cities. I chose to include the adjoining roads as well as there may be instances where roads are on the boundary of the city but not in which would alter the estimated road distance quite drastically in some cases. Creating a dissolved polygon works better than 6,000 individual ones as some roads may be clipped in unexpected ways, this also helps with the speed and optimisation of the python file.

In [ ]:
#dissolve poylgons
city_polygons = lsoa_cleaned.dissolve()
city_polygons.explore(width="80%", height="60%")

city_polygons.plot()
print('city_polygons.crs:', city_polygons.crs)

In [ ]:
#clip Road network by City_Dissolved

#Specify lines, and mask to clip by
roads_clipped = gpd.clip(roads, city_polygons)

roads_clipped.explore(width="80%", height="60%")

In [ ]:
roads_clipped.head()

In [ ]:
#Copy the road network and make an ONLY Primary Roads layer
#filter to just motorways and primary A roads
primary_roads = roads_clipped[roads_clipped["class"].str.contains("Motorway") | (roads_clipped["class"].str.contains("A Road") & (roads_clipped["primary"] == ('true')))]

#primary_roads = roads_clipped[roads_clipped["class"].str.contains("Motorway") & (roads_clipped["primary"] == ('true') | (roads_clipped["class"].str.contains("A Road") & (roads_clipped["primary"] == ('true'))))]

#Keeping certain columns removes the geometry field, so drop specific columns instead
primary_roads = primary_roads.drop(['path', 'layer','loop','startNode','endNode','fictitious','identifier','name1','name1_lang','name2','name2_lang','nameTOID','numberTOID','structure'], axis=1, )

primary_roads.explore(width="80%", height="60%")

My original plan was to upload the crimes excel sheet and convert into a geodata frame using the Fiona package however I experienced problems with the folium map section (Data Ninjas, 2021). So, I opted instead to create the shapefile as above.

In [ ]:
#check dtypes
#crimes.head()
#crimes.dtypes

# Create coordiantes column, specify as a list and concatinate the Lat, Long
#crimes['coordinates'] = crimes[['Longitude','Latitude']].values.tolist()
#crimes.head()

#from shapely.geometry import Point
# Add point infront of coords list
#crimes['coordinates'] = crimes['coordinates'].apply(Point)
#crimes.head()

#find dataframe type
#type(crimes)

# use column "coords" as geometry, check dataframe type
#crime_points = gpd.GeoDataFrame(crimes, geometry = 'coordinates')
#type(crime_points)

#crime_points.plot()
#crime_points.explore()

#crime_points = crime_points.set_crs('epsg:4326')
#crime_points.crs
#crime_points = crime_points.to_crs('epsg:27700')
#crime_points.crs

#crime_points.to_file('export.shp')
#needd to export firrst??? gdf
#crimes_plotted = gpd.read_file("export.shp")

#crimes_plotted.explore()

#TypeError: 'NoneType' object is not subscriptable
#<folium.folium.Map at 0x7a25554564a0>

In [ ]:
crimes_merged.head()

In [ ]:
# Clip burglary points to just Cities of interest, this also remove any null geometry
crimes_clipped = gpd.clip(crimes_merged, city_polygons)

# Locate only Burglary Data
burglary = crimes_clipped.loc[crimes_clipped['Crime_type']=='Burglary']

burglary.explore(width="80%", height="60%")

I was also going to calculate the distance between points along a network to gain a true distance from burglary to primary road but due to a problem of conflicting tool instructions and package version instructions I could not get the get_nearest_node command to work (automating-gis-processes, 2017). To get around this I chose to use the Euclidian distance (do-me, 2023), although this would not be a true account it could still be used as a reasonable estimate of distance for this use case.

Utilising the spatial join command against burglary points and the primary road start points I created my new data frame, using a maximum distance tolerance of 2000m to reduce any outlier distances as most of the city would be covered within 2000m. I also calculated some additional statistics using this distance information to join onto the LSOA data frame using the group-by and aggregate functions to summarise the count of burglaries and the average distance to the primary road per output area.

In [ ]:
road_start_points.head()

In [ ]:
# find nearest primary road point from burglary point
crime_dist = gpd.sjoin_nearest(burglary, road_start_points, how='inner', max_distance = 2000, distance_col="distance", lsuffix="left", rsuffix="right")

In [ ]:
crime_dist.info()

In [ ]:
# only keep LSOA and distance columns
crime_dist = crime_dist[['LSOA_name', 'distance']]
#add copied column of distance for avg distance calculation
crime_dist['avg_dist'] = crime_dist['distance']

# Create new df for count of crimes in LSOA
crime_count = crime_dist.groupby('LSOA_name').count().reset_index().rename(columns={'distance':'count_of_burg'})
# Create group column for avergae distance
crime_count1 = crime_dist.groupby('LSOA_name').agg({'distance':'mean'}).reset_index().rename(columns={'distance':'avg_distance'})
#Merge results together to form one reocrd per LSOA with count and avg.
crime_graph = pd.merge(crime_count, crime_count1, how='inner', left_on='LSOA_name', right_on='LSOA_name')
#Remove duplicate column
crime_graph = crime_graph.drop(columns=['avg_dist'])

crime_graph.head()

In [ ]:
# Merge LSOA_cleaned data onto crime count and avg ditance data
crime_census = pd.merge(lsoa_cleaned, crime_graph, how='left', left_on='LSOA21NM', right_on='LSOA_name')
# Merge the LSOA and crime data onto Census data
crime_census = pd.merge(crime_census, census, how='left', left_on='LSOA21NM', right_on='geography')

crime_census.explore(width="80%", height="60%")

In [ ]:
# Make a centroid layer of LSOAs
# copy GeoDataFrame
#lsoa_centroid = crime_census.copy()
# change geometry to centroid of LSOA
#lsoa_centroid['geometry'] = lsoa_centroid['geometry'].centroid
#lsoa_centroid.head()

In [ ]:
#lsoa_cleaned = LSOA.loc[LSOA["LSOA21NM"].str.contains("Leeds | Bradford | Hull | Sheffield")]  #already have????

For the last part of data processing, I split the data frames into their respective city limits Leeds, Sheffield, Bradford. Hull was left out of the final visualisation as there were no burglary points visualised in the Hull area.

In [ ]:
crime_census.head()

In [ ]:
#Filter LSOA data and make new dataframe for each city, to aid in map image scaling
spatial_leeds = crime_census.loc[crime_census["LSOA21NM"].str.contains("Leeds")]
spatial_bradford = crime_census.loc[crime_census["LSOA21NM"].str.contains("Bradford")]
spatial_sheffield = crime_census.loc[crime_census["LSOA21NM"].str.contains("Sheffield")]

# Clip Primary Roads to make new dfs for each city
primary_leeds = gpd.clip(primary_roads, spatial_leeds)
primary_bradford = gpd.clip(primary_roads, spatial_bradford)
primary_sheffield = gpd.clip(primary_roads, spatial_sheffield)

Visualisation:

Using Spearman’s correlation, I found that the average distance has an overall weak relationship to the number of burglaries but a moderate relationship between number of burglary and both one- and two-bedroom accommodation. These are found on the outskirts of the city, however there are also some accommodations within. For this visual I used a mask to cover off the other half of the data to tidy up the map. I used the RdBu colour palette as using colour brewer as a reference of different possible palettes for my graphs and this one is a colourblind friendly choice and is best used as a diverging colour scheme (Aptech, 2024).

In [ ]:
# Calculate Spearman's rank correlation
spearmans = crime_census[['count_of_burg',
       'avg_distance',
       'Population Density: km2',
       'Household size: <3 people in household',
      'Household size: +3 people in household','Number of bedrooms: 1 bedroom','Number of bedrooms: 2 bedrooms',
      'Number of bedrooms: 3 bedrooms','Number of bedrooms: 4 or more bedrooms']].corr(method = 'spearman')
spearmans

In [ ]:
# define plot size
fig,ax = plt.subplots(figsize=(8,8))

# define mask to apply to upper right hand corner of the plot
data_to_mask = np.triu(np.ones_like(spearmans))

# plot a heatmap of the correlation dataframe
sns.heatmap(spearmans,
            # annotate so spearman's rank correlation values are displayed on the squares
            annot=True,
            # define colourmap
            cmap='RdBu',
            # define value of minimum colour on cbar
            vmin=-1,
            # define value of maximum colour on cbar
            vmax=1,
            # add the mask
            mask=data_to_mask,
            # add a label to the cbar
            cbar_kws={'label': "Spearman's Rank correlation"},
            # plot on the axis we defined
            ax=ax)

# Set axis labels
ax.set(title ="Correlation Census for Burglary's near Primary Roads" );

#for online google colab download
from google.colab import files
plt.savefig("Correlation Census for Burglary's near Primary Roads.png")
files.download("Correlation Census for Burglary's near Primary Roads.png")

In [ ]:
sns.histplot(crime_dist, x="distance")
plt.xlabel("Distance from Primary Road (m)")
plt.ylabel("Count of Burglarys")
plt.show()

For my visuals I chose a green colour for the joint plot scatter graph, this was going to be blue but I though the contrast between the dark blue and light blue colours made it difficult to clearly see the results so I opted for a green instead, a single colour tone will not affect visually impaired users. Using the bar charts, you can see a cluster near the 250m mark which steadily decreases with greater distances from the primary roads.
Better visualised by the hex plot graph it is far easier to spot clusters of significance in darker blue where there is lots of overlap of results, which was very difficult to identify on the previous graph. The correlation shows a larger concentration of burglary’s along the 250m mark which carries over to 500m and steadily declines from the 750m mark. I also renamed the axis labels to be more intuitive for the users who will be the Police force and crime analysts giving them a clear and easily read graph.


In [ ]:
# joint plot, specify colour and marker type
sns.jointplot(x='avg_distance',y='count_of_burg', color='green', kind ='scatter',data=crime_graph,
            height=10, marker='*');
# add label axis
plt.xlabel("Average distance from Primary Road (m)")
plt.ylabel("Count of Burglarys per Output Area")
plt.title("LSOA Burglarys by Average Distance to Primary road")
plt.show()

In [ ]:
# plot a hex plot
sns.jointplot(x='avg_distance',y='count_of_burg', kind ='hex',data=crime_graph);
# add label axis
plt.xlabel("Average distance from Primary Road (m)")
plt.ylabel("Count of Burglarys per Output Area")
plt.title("Output Area Burglary's by Average Distance to Primary road", y=1.2, fontsize = 12)
plt.show()

Graphs:

For the final visual I plotted the spatial representation of burglary and added the primary road layer relative to the output areas allowing for easy identification of hotspots and clusters of data. This is so the police and crime analysts will instinctively be able to tell areas of focus against the road network their familiar with to reduce the crime rate. I used the ‘YlOrRd’ colour palette as this is both colour blind friendly (Aptech, 2024) and you can easily distinguish the different level of burglary rates by using the ‘temperature’ of the colour.

On the maps, you can see a high concentration near the centre of Leeds which is completely encircled by primary roads and other clusters directly in between these prime roads. Lighter colours are seen farther from the prime roads. This description can be effectively applied to all the other maps as well.


In [ ]:
# create a figure with three subplots (maps)
f,ax = plt.subplots(3,1, figsize=(36,24))

# plot leeds in subplot 1 and leeds roads
base1 = spatial_leeds.plot(ax=ax[0], column ='count_of_burg', cmap='YlOrRd', legend=True)
primary_leeds.plot(ax=base1,color='red')

# plot bradford in subplot 2 and bradford roads
base2 = spatial_bradford.plot(ax=ax[1], column ='count_of_burg', cmap='YlOrRd', legend=True)
primary_bradford.plot(ax=base2,color='red')

# plot sheffield in subplot 3 and sheffield roads
base3 = spatial_sheffield.plot(ax=ax[2], column ='count_of_burg', cmap='YlOrRd', legend=True)
primary_sheffield.plot(ax=base3,color='red')

# give subplot 1 title
# Use ax[0] to speficy plots
ax[0].set_title("Leeds Burglarys by LSOA 2023-2024")
# give subplot 2 title
ax[1].set_title("Bradford Burglarys by LSOA 2023-2024")
# give subplot 3 title
ax[2].set_title("Sheffield Burglarys by LSOA 2023-2024")

# make axis invisible
ax[0].set_axis_off()
ax[1].set_axis_off()
ax[2].set_axis_off()

# show figure
plt.show()

#for online google colab download
#from google.colab import files

#test = plt.figure()
#plt.plot([1, 2, 3])
#plt.savefig('test.pdf')

#files.download('test.pdf')

In [ ]:
# Download new CSV data
# for online google colab download
from google.colab import files
crime_census.to_csv('crime_census_export.csv', sep=',', index=False, encoding='utf-8')
files.download('crime_census_export.csv')

References:

Aptech. 2024. Color Brewer Palettes. [Online]. [Accessed 28 April 2024]. Available from: https://www.aptech.com/releases/gauss18/graphics-updates/color-brewer-palettes/

Tableau. 2024. 5 tips on designing colour-blind friendly visualisation. [Online]. [Accessed 26 April 2024]. Available from: https://www.tableau.com/en-gb/blog/examining-data-viz-rules-dont-use-red-green-together

Data Ninjas. 2021. Create a shapefile form a .csv with Python, web map it and reproject it. [Online]. [Accessed 20 April 2024]. Available from: https://www.youtube.com/watch?v=RAn8Lx0qrks

jeremysze. 2019. Network Distance. [Online]. [Accessed 29 April 2024]. Available from: https://jeremysze.github.io/GIS_exploration/build/html/networkdistance_osm.html

automating-gis-processes. 2017. Network analysis in Python. [Online]. [Accessed 29 April 2024]. Available from: https://automating-gis-processes.github.io/2017/lessons/L7/network-analysis.html

Aric A. Hagberg, Daniel A. Schult and Pieter J. Swart, “Exploring network structure, dynamics, and function using NetworkX”, in Proceedings of the 7th Python in Science Conference (SciPy2008), Gäel Varoquaux, Travis Vaught, and Jarrod Millman (Eds), (Pasadena, CA USA), pp. 11–15, Aug 2008

do-me. 2023. As of v0.10.0 geopandas supports sjoin_nearest natively. 13 October 2023. Finding nearest point in other GeoDataFrame using GeoPandas. [Online]. [Accessed 26 April 2024]. Available from: https://gis.stackexchange.com/questions/222315/finding-nearest-point-in-other-geodataframe-using-geopandas

nomis. 2021. TS006 Population Density. Office for National Statistics. [Online]. [Accessed 7 April 2024]. Available from: https://www.nomisweb.co.uk/sources/census_2021_bulk

nomis. 2021. TS0017 Household Size. Office for National Statistics. [Online]. [Accessed 7 April 2024]. Available from: https://www.nomisweb.co.uk/sources/census_2021_bulk

nomis. 2021. TS050 Number of Bedrooms. Office for National Statistics. [Online]. [Accessed 7 April 2024]. Available from: https://www.nomisweb.co.uk/sources/census_2021_bulk

Geoportal Statistics. 2023. Output Areas (December 2021) Boundaries EW BGC V2. ONS Geography. [Online]. [Accessed 7 April 2024]. Available from: https://geoportal.statistics.gov.uk/datasets/6beafcfd9b9c4c9993a06b6b199d7e6d_0/explore?location=53.594232%2C-1.209805%2C9.00

data.police. no date. Data Download. data.police. [Online]. [Accessed 9 April 2024]. Available from: https://data.police.uk/data/

digimap. 2023. Highways – Roads (GB). Digimap Edina. [Online]. [Accessed 9 April 2024]. Available from: https://digimap.edina.ac.uk/roam/map/os

